In [1]:
library(tidyr)
library(dplyr)
library(corrplot)


lifeData <- read.csv("D:/Code/BIOSTAT682/Life Expectancy_v4.csv")

dim(lifeData)
head(lifeData)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


corrplot 0.92 loaded



[1] 1646   18

,Country,Year,Status,Life.expectancy,Adult.Mortality,Alcohol,Hepatitis.B,Measles,BMI,Polio,Total.expenditure,Diphtheria,HIV.AIDS,GDP,Population,thinness.1.19.years.,Income.composition.of.resources,Schooling
,<chr>,<int>,<chr>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Afghanistan,2015,Developing,65.0,263,0.01,65,1154,19.1,6,8.16,65,0.1,0.005842592,33736494,17.2,0.479,10.1
2,Afghanistan,2014,Developing,59.9,271,0.01,62,492,18.6,58,8.18,62,0.1,0.006126965,32758200,17.5,0.476,10.0
3,Afghanistan,2013,Developing,59.9,268,0.01,64,430,18.1,62,8.13,64,0.1,0.006317450,31731688,17.7,0.470,9.9
4,Afghanistan,2012,Developing,59.5,272,0.01,67,2787,17.6,67,8.52,67,0.1,0.006699590,36969580,17.9,0.463,9.8
5,Afghanistan,2011,Developing,59.2,275,0.01,68,3013,17.2,68,7.87,68,0.1,0.006353723,29785990,18.2,0.454,9.5
6,Afghanistan,2010,Developing,58.8,279,0.01,66,1989,16.7,66,9.20,66,0.1,0.005533289,28831670,18.4,0.448,9.2


In [2]:
lifeData_train <- lifeData %>% filter(Year < 2013)
lifeData_test <- lifeData %>% filter(Year >= 2013)
dim(lifeData_train)
dim(lifeData_test)

[1] 1386   18

[1] 260  18

In [3]:
library(R2jags)

status = rep(0,nrow(lifeData_train))

for (i in 1:nrow(lifeData_train)) {
  status[i] = ifelse(lifeData_train$Status[i]=="Developing",1,0)
}
mortality = lifeData_train$"Adult.Mortality"
alcohol = lifeData_train$"Alcohol"
hepB = lifeData_train$"Hepatitis.B"
measles = lifeData_train$"Measles"
BMI = lifeData_train$"BMI"
polio = lifeData_train$"Polio"
totalExp = lifeData_train$"Total.expenditure"
dipth = lifeData_train$"Diphtheria"
hivaids = lifeData_train$"HIV.AIDS"
gdp = lifeData_train$"GDP"
pop = lifeData_train$"Population"
thin1019 = lifeData_train$"thinness.1.19.years"
incomeComp = lifeData_train$"Income.composition.of.resources"
school = lifeData_train$"Schooling"
y = lifeData_train$"Life.expectancy"
N = length(lifeData_train$Country) #go through each point
J = length(unique(lifeData_train$Country)) #go through each country

# encode country
indicesDict = c(unique(lifeData_train$Country))
indices = rep(0,N)
for (i in 1:N) {
  indices[i] = which(indicesDict==lifeData_train$Country[i])
}

Warning message:
"package 'R2jags' was built under R version 4.2.1"
Loading required package: rjags

Warning message:
"package 'rjags' was built under R version 4.2.1"
Loading required package: coda

Warning message:
"package 'coda' was built under R version 4.2.1"
Linked to JAGS 4.3.1

Loaded modules: basemod,bugs


Attaching package: 'R2jags'


The following object is masked from 'package:coda':

    traceplot




## Model 1

In [27]:
life_expectancy.model.1 = function()
{
  for (i in 1:N) {
     y[i] ~ dnorm(mu[indices[i],i],tau) #could do tau*k bc precision
     #indices[i] gets the cluster number
     mu[indices[i],i] <- 
          betay[indices[i],1] # intercept
          + betay[indices[i],2]*status[i] 
          + betay[indices[i],3]*mortality[i] 
          + betay[indices[i],4]*alcohol[i] 
          + betay[indices[i],5]*hepB[i] 
          + betay[indices[i],6]*measles[i] 
          + betay[indices[i],7]*BMI[i] 
          + betay[indices[i],8]*polio[i] 
          + betay[indices[i],9]*totalExp[i] 
          + betay[indices[i],10]*dipth[i] 
          + betay[indices[i],11]*hivaids[i] 
          + betay[indices[i],12]*gdp[i] 
          + betay[indices[i],13]*pop[i] 
          + betay[indices[i],14]*thin1019[i] 
          + betay[indices[i],15]*incomeComp[i] 
          + betay[indices[i],16]*school[i] 
  }
  for (j in 1:J) {
    for (k in 1:16) {
      betay[j,k] ~ dnorm(beta[k],tau2)
    }
  }      
  for (k in 1:16) {
    beta[k] ~ dnorm(0,1e-6)
  }
  tau ~ dgamma(1e-6,1e-6)
  tau2 ~ dgamma(1e-6,1e-6)
}

In [23]:
save.parms.1 = c("beta","betay")
life_expectancy.data.1 = c("y","mortality","alcohol", "status",
                            "hepB","measles","BMI","polio","totalExp",
                            "dipth","hivaids","gdp","pop","thin1019",
                            "incomeComp","school","N","J","indices")                           

In [28]:
life_expectancy.out.1 = jags(data=life_expectancy.data.1,parameters.to.save=save.parms.1,model.file=life_expectancy.model.1,
                           n.chains=3,n.iter=10000,n.burnin=2000,n.thin=1)
print(life_expectancy.out.1, digits = 3)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 1386
   Unobserved stochastic nodes: 2114
   Total graph size: 42302

Initializing model

Inference for Bugs model at "F:/temp/RtmpgT1z7k/model2228422713b.txt", fit using jags,
 3 chains, each with 10000 iterations (first 2000 discarded)
 n.sims = 24000 iterations saved
               mu.vect sd.vect     2.5%      25%      50%      75%    97.5%
beta[1]         -0.068   0.071   -0.167   -0.164   -0.033    0.001    0.006
beta[2]          0.036   0.125   -0.128   -0.102    0.038    0.192    0.202
beta[3]          0.013   0.005    0.008    0.009    0.014    0.016    0.026
beta[4]         -0.166   0.147   -0.386   -0.383   -0.069   -0.065   -0.031
beta[5]         -0.008   0.011   -0.021   -0.017   -0.013    0.006    0.008
beta[6]          0.000   0.002   -0.002    0.000    0.000    0.000    0.002
beta[7]         -0.139   0.038   -0.190   -0.187   -0.139   -0.107   -0.

In [4]:
save.parms.0 = c("beta")
life_expectancy.data.0 = c("y","mortality","alcohol", "status",
                            "hepB","measles","BMI","polio","totalExp",
                            "dipth","hivaids","gdp","pop","thin1019",
                            "incomeComp","school","N")    

life_expectancy.model.0 = function()
{
  for (i in 1:N) {
     y[i] ~ dnorm(mu[i],tau) #could do tau*k bc precision
     #indices[i] gets the cluster number
     mu[i] <- 
          beta[1] # intercept
          + beta[2]*status[i] 
          + beta[3]*mortality[i] 
          + beta[4]*alcohol[i] 
          + beta[5]*hepB[i] 
          + beta[6]*measles[i] 
          + beta[7]*BMI[i] 
          + beta[8]*polio[i] 
          + beta[9]*totalExp[i] 
          + beta[10]*dipth[i] 
          + beta[11]*hivaids[i] 
          + beta[12]*gdp[i] 
          + beta[13]*pop[i] 
          + beta[14]*thin1019[i] 
          + beta[15]*incomeComp[i] 
          + beta[16]*school[i] 
  }
  for (k in 1:16) {
    beta[k] ~ dnorm(0,1e-6)
  }
  tau ~ dgamma(1e-6,1e-6)
}

In [5]:
life_expectancy.out.0 = jags(data=life_expectancy.data.0,parameters.to.save=save.parms.0,model.file=life_expectancy.model.0,
                           n.chains=3,n.iter=10000,n.burnin=2000,n.thin=1)
print(life_expectancy.out.0, digits = 3)

module glm loaded



Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 1386
   Unobserved stochastic nodes: 17
   Total graph size: 30440

Initializing model

Inference for Bugs model at "F:/temp/RtmpKGpu2M/model40207a3a7218.txt", fit using jags,
 3 chains, each with 10000 iterations (first 2000 discarded)
 n.sims = 24000 iterations saved
          mu.vect sd.vect     2.5%      25%      50%      75%    97.5%  Rhat
beta[1]    63.237   0.931   61.388   62.616   63.242   63.872   65.042 1.001
beta[2]    -0.228   0.288   -0.796   -0.422   -0.228   -0.036    0.340 1.001
beta[3]    -0.044   0.001   -0.047   -0.045   -0.044   -0.043   -0.042 1.001
beta[4]    -0.029   0.029   -0.085   -0.048   -0.029   -0.010    0.028 1.001
beta[5]    -0.004   0.005   -0.014   -0.008   -0.004   -0.001    0.006 1.001
beta[6]     0.000   0.000    0.000    0.000    0.000    0.000    0.000 1.000
beta[7]     0.050   0.008    0.034    0.045    0.050    0.056    0